# Внесение размеченных масок из CVAT на исходные изображения railsem

- нужно распарсить координаты bboxes
- распарситьь маски внутри bboxes
- нужно по координатам bboxes на исходное изображенеи перенести маску и так для всех изображений

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import shapely.geometry as geom

import json

import os
from pathlib import Path
import shutil

from pprint import pprint
from tqdm import tqdm
import locale

import shapely.geometry as geom
import numpy as np
from ultralytics.data.utils import polygon2mask

- чтение json файлов(2)
- вытаскивание координат из rs
- coco to mask froamt polytgon check

In [2]:
rs_annot = 'annotation_rs/jsons/rs19_val/rs07854.json'
classnames = ['switch-left', 'switch-right', 'switch-unknow']
coco_annot = 'annotation_mine/instances_Validation.json'
coco_result_annot = 'results/annot_coco_big.json'
rs_true_img_dir = 'rs_true/jpgs/rs19_val'


rs_annot_dir = 'annotation_rs/jsons/rs19_val'
rs_annot_list = os.listdir('annotation_rs/jsons/rs19_val')
rs_imgs = os.listdir(rs_true_img_dir)

In [3]:
#создаем словарь имен файлов и координат bboxes true
bboxes_scale_dict = {}
img_names = []
for filename in rs_annot_list:
    with open(rs_annot_dir + '/' + filename) as file:
        data = json.loads(file.read())

    increase_bboxes = 2

    wh_inc = 1 + increase_bboxes / 100
    xy_inc = 1 - increase_bboxes / 100

    for idx, obj in enumerate(data['objects']):
        if obj['label'] in classnames:
            try:
                x, y, w, h = obj['boundingbox']

                if increase_bboxes != 0:
                    x, y = int(xy_inc * x), int(xy_inc * y)
                    w, h = int(wh_inc * w), int(wh_inc * h)

                    firstname = filename.split(".")[0]
                    name = f'{firstname}_{idx}'
                    img_names.append(firstname)
                    bboxes_scale_dict[name] = (x, y)
                    # print(x, y)
            except:
                continue


print(len(bboxes_scale_dict))

4043


In [4]:
bboxes_scale_dict['rs05840_10']

(727, 560)

# TODO: 
- после того, как размечу и скину сюда всю аннотацию coco к 4043 ищображениям прогнатьвсе через этот код и должно быть + - это число полигонов
- перенести эти полигоны по именам больших изображений и состьавить свою аннотацию в формате coco(у каждого изображения может быть несколько аннотаций)

In [5]:
#достаем полигоны по имени файла rs_true в аннотации coco
with open(coco_annot) as file:
    data = file.read()
dic = json.loads(data)



polygons = {}
for img_name in img_names:
    img_ids = []
    for img_obj in dic['images']:
        if img_name in img_obj['file_name']:
            # img_ids.append(img_obj['id'])
            for cur_annot in dic['annotations']:
                # for id in img_ids:
                if cur_annot['image_id'] == img_obj['id']:
                    polygons[img_obj['file_name'].split('.')[0]] = cur_annot['segmentation'][0]


# # #скейлим один полигон
# polygon_base = polys[0]
# polygon_scaled = []
# for idx in range(len(polygon_base)):
#   if idx % 2 == 0:
#     polygon_scaled.append(polygon_base[idx] + bboxes_dict['coord1'][0])
#   else:
#     polygon_scaled.append(polygon_base[idx] + bboxes_dict['coord1'][1])


# print(polygon_scaled)
# print(polygon_scaled2)
# print(len(polygons))
              
#5840 - 8496? 1995
len(polygons)

3155

In [7]:
result_bboxes = {}
for key, value in polygons.items():
    polygon_scaled = []
    for idx in range(len(value)):
        if idx % 2 == 0:
            polygon_scaled.append(value[idx] + bboxes_scale_dict[key][0])   #посмотреть на обучении мб придется поменять местами 1 и 0
        else:
            polygon_scaled.append(value[idx] + bboxes_scale_dict[key][1])
    result_bboxes[key] = polygon_scaled

In [8]:
result_bboxes

{'rs00007_0': [1214.72,
  670.29,
  1221.0,
  692.17,
  1220.29,
  726.37,
  1209.97,
  754.66,
  1222.13,
  755.0,
  1356.0,
  755.0,
  1356.0,
  735.99,
  1281.1,
  662.0,
  1211.21,
  662.0],
 'rs00007_11': [1176.95,
  620.89,
  1187.15,
  626.25,
  1194.23,
  631.26,
  1198.55,
  637.65,
  1202.18,
  644.05,
  1258.34,
  637.65,
  1251.43,
  630.05,
  1245.21,
  624.35,
  1241.86,
  620.0,
  1194.06,
  620.0],
 'rs00007_17': [736.3, 828.92, 921.0, 850.72, 921.0, 759.0, 824.72, 759.0],
 'rs00007_18': [1051.0, 851.0, 1051.0, 745.0, 838.0, 745.0, 838.0, 851.0],
 'rs00007_2': [1014.77,
  605.0,
  1021.15,
  608.62,
  1026.34,
  615.01,
  1025.65,
  623.83,
  1022.02,
  629.7,
  1010.79,
  639.55,
  1004.05,
  643.35,
  1066.77,
  651.13,
  1103.0,
  606.94,
  1103.0,
  605.0],
 'rs00007_3': [883.62,
  597.83,
  906.95,
  591.09,
  921.81,
  596.62,
  932.87,
  600.59,
  937.19,
  603.7,
  944.44,
  606.99,
  948.0,
  610.08,
  948.0,
  625.0,
  858.0,
  625.0,
  858.0,
  622.13,
  867.

In [9]:
with open(coco_annot) as file:
    data = file.read()
dic = json.loads(data)


new_img_names = []
for item in polygons.keys():
    new_img_names.append(item.split('_')[0])

new_img_names = set(new_img_names)
new_img_names

{'rs08117',
 'rs02347',
 'rs03291',
 'rs04130',
 'rs00266',
 'rs04251',
 'rs02853',
 'rs00879',
 'rs08496',
 'rs01702',
 'rs01440',
 'rs06585',
 'rs06823',
 'rs02691',
 'rs07742',
 'rs02327',
 'rs03505',
 'rs01302',
 'rs04232',
 'rs07547',
 'rs08392',
 'rs02912',
 'rs03465',
 'rs06773',
 'rs06858',
 'rs00633',
 'rs00309',
 'rs00087',
 'rs03342',
 'rs07101',
 'rs06965',
 'rs08201',
 'rs02208',
 'rs01492',
 'rs07972',
 'rs06540',
 'rs07007',
 'rs03333',
 'rs04060',
 'rs07804',
 'rs08397',
 'rs08137',
 'rs02341',
 'rs07666',
 'rs03281',
 'rs03471',
 'rs08231',
 'rs02572',
 'rs06653',
 'rs06895',
 'rs05859',
 'rs00116',
 'rs02136',
 'rs06289',
 'rs01402',
 'rs02717',
 'rs01384',
 'rs02957',
 'rs07969',
 'rs04039',
 'rs02059',
 'rs04317',
 'rs01905',
 'rs08089',
 'rs03618',
 'rs03576',
 'rs08450',
 'rs03195',
 'rs00925',
 'rs06638',
 'rs06975',
 'rs06649',
 'rs01912',
 'rs01642',
 'rs02442',
 'rs01967',
 'rs00387',
 'rs00992',
 'rs07315',
 'rs04102',
 'rs04170',
 'rs03484',
 'rs01411',
 'rs

In [10]:
with open(coco_result_annot) as file:
    data = file.read()
dic = json.loads(data)

images = []


for idx, img_name in enumerate(new_img_names):
    images.append(
        {
            "id": idx,
            "width": 1920,
            "height": 1080,
            "file_name": img_name + '.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        },
    )
dic['images'] = images

        {
            "id": 109,
            "image_id": 119,
            "category_id": 1,
            "segmentation": [
                [
                    7.44,
                    47.21,
                    7.94,
                    32.01,
                    11.24,
                    21.71,
                    13.94,
                    8.91,
                    16.64,
                    7.31,
                    38.54,
                    7.41,
                    49.14,
                ]
            ],
            "area": 2195.0,
            "bbox": [
                7.44,
                6.51,
                66.39,
                41.49
            ],
            "iscrowd": 0,
            "attributes": {
                "occluded": false
            }
        },

# Добавляем правильные bounding boxes к аннотациям

In [11]:
#создаем словарь имен файлов и координат bboxes true
bboxes = {}
img_names = []
for filename in rs_annot_list:
    with open(rs_annot_dir + '/' + filename) as file:
        data = json.loads(file.read())

    increase_bboxes = 2

    wh_inc = 1 + increase_bboxes / 100
    xy_inc = 1 - increase_bboxes / 100

    for idx, obj in enumerate(data['objects']):
        if obj['label'] in classnames:
            try:
                x, y, w, h = obj['boundingbox']

                if increase_bboxes != 0:
                    x, y = int(xy_inc * x), int(xy_inc * y)
                    w, h = int(wh_inc * w), int(wh_inc * h)

                    firstname = filename.split(".")[0]
                    name = f'{firstname}_{idx}'
                    img_names.append(firstname)
                    bboxes[name] = [x, y, w, h]
                    # print(x, y)
            except:
                continue


pprint(bboxes)

{'rs00007_0': [1192, 662, 1356, 755],
 'rs00007_11': [1172, 620, 1291, 663],
 'rs00007_17': [706, 759, 921, 851],
 'rs00007_18': [838, 745, 1051, 851],
 'rs00007_2': [995, 605, 1103, 655],
 'rs00007_3': [858, 578, 948, 625],
 'rs00019_0': [936, 559, 998, 590],
 'rs00024_0': [791, 656, 964, 756],
 'rs00025_0': [1198, 512, 1303, 550],
 'rs00029_0': [933, 484, 1051, 549],
 'rs00035_0': [878, 654, 974, 700],
 'rs00039_0': [731, 465, 1145, 646],
 'rs00046_1': [1176, 428, 1282, 477],
 'rs00047_0': [946, 551, 1028, 595],
 'rs00057_4': [373, 778, 656, 918],
 'rs00059_2': [875, 614, 952, 675],
 'rs00059_5': [878, 593, 936, 634],
 'rs00066_2': [914, 546, 999, 599],
 'rs00068_1': [876, 596, 954, 642],
 'rs00074_1': [428, 583, 581, 664],
 'rs00074_2': [569, 510, 705, 565],
 'rs00074_3': [692, 465, 778, 499],
 'rs00074_4': [693, 437, 755, 467],
 'rs00080_0': [935, 702, 1081, 783],
 'rs00084_0': [686, 696, 792, 742],
 'rs00084_1': [731, 667, 825, 710],
 'rs00084_11': [847, 735, 1063, 877],
 'rs00084

In [12]:
annotations = []
for idx, img_name in enumerate(polygons.keys()):
    #frame_10
    for img_object in dic['images']:
        if img_name.split('_')[0] in img_object['file_name']:
            mask = result_bboxes.get(img_name)
            bbox = bboxes.get(img_name)
            annotations.append(
                        {
                            "id": idx,
                            "image_id": img_object['id'],
                            "category_id": 1,
                            "segmentation": [
                                mask
                            ],
                            "area": 2195.0,
                            "bbox": bbox,
                            "iscrowd": 0,
                            "attributes": {
                                "occluded": False
                            }
                        },
            )

In [13]:
polygons.keys()

dict_keys(['rs00007_0', 'rs00007_11', 'rs00007_17', 'rs00007_18', 'rs00007_2', 'rs00007_3', 'rs00019_0', 'rs00024_0', 'rs00025_0', 'rs00029_0', 'rs00035_0', 'rs00039_0', 'rs00046_1', 'rs00047_0', 'rs00059_2', 'rs00059_5', 'rs00066_2', 'rs00068_1', 'rs00074_1', 'rs00074_2', 'rs00074_3', 'rs00074_4', 'rs00080_0', 'rs00084_0', 'rs00084_1', 'rs00084_11', 'rs00084_12', 'rs00084_6', 'rs00087_0', 'rs00091_1', 'rs00092_1', 'rs00096_2', 'rs00097_0', 'rs00097_1', 'rs00103_0', 'rs00107_0', 'rs00109_0', 'rs00110_2', 'rs00110_3', 'rs00111_0', 'rs00116_0', 'rs00125_0', 'rs00125_1', 'rs00126_0', 'rs00127_0', 'rs00127_1', 'rs00128_5', 'rs00131_0', 'rs00136_0', 'rs00144_0', 'rs00146_0', 'rs00149_0', 'rs00150_11', 'rs00155_1', 'rs00155_2', 'rs00160_1', 'rs00160_2', 'rs00160_3', 'rs00163_0', 'rs00163_1', 'rs00166_2', 'rs00174_0', 'rs00174_12', 'rs00174_13', 'rs00174_3', 'rs00174_4', 'rs00174_5', 'rs00175_2', 'rs00186_0', 'rs00186_1', 'rs00188_2', 'rs00193_0', 'rs00193_1', 'rs00193_2', 'rs00193_3', 'rs001

In [14]:
len(annotations)

3155

In [15]:
dic['annotations'] = annotations

In [16]:
dic['images']

[{'id': 0,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs08117.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 1,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs02347.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 2,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs03291.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 3,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs04130.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 4,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs00266.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 5,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs04251.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 6,
  'width': 1920,
  'height': 1080,
  'file_name': 'rs02853.jpg',
  'lic

In [17]:
dic['annotations']

[{'id': 0,
  'image_id': 1595,
  'category_id': 1,
  'segmentation': [[1214.72,
    670.29,
    1221.0,
    692.17,
    1220.29,
    726.37,
    1209.97,
    754.66,
    1222.13,
    755.0,
    1356.0,
    755.0,
    1356.0,
    735.99,
    1281.1,
    662.0,
    1211.21,
    662.0]],
  'area': 2195.0,
  'bbox': [1192, 662, 1356, 755],
  'iscrowd': 0,
  'attributes': {'occluded': False}},
 {'id': 1,
  'image_id': 1595,
  'category_id': 1,
  'segmentation': [[1176.95,
    620.89,
    1187.15,
    626.25,
    1194.23,
    631.26,
    1198.55,
    637.65,
    1202.18,
    644.05,
    1258.34,
    637.65,
    1251.43,
    630.05,
    1245.21,
    624.35,
    1241.86,
    620.0,
    1194.06,
    620.0]],
  'area': 2195.0,
  'bbox': [1172, 620, 1291, 663],
  'iscrowd': 0,
  'attributes': {'occluded': False}},
 {'id': 2,
  'image_id': 1595,
  'category_id': 1,
  'segmentation': [[736.3,
    828.92,
    921.0,
    850.72,
    921.0,
    759.0,
    824.72,
    759.0]],
  'area': 2195.0,
  'bbox

In [18]:
with open(r"C:\Users\ivano\Документы\GitHub\Rail_segm\code\data_prepare\results\annot_coco_big_finall.json", "w", encoding="utf-8") as file:
    json.dump(dic, file)

In [91]:
dic['images'][0]['file_name']

'rs06995.jpg'

In [5]:
#Загрузить изображение


image = cv2.imread(r"C:\Users\ivano\Документы\GitHub\Rail_segm\code\data_prepare\rs_true\jpgs\rs19_val\rs05840.jpg")

polygon_scaled =[
                    731.0,
                    560.0,
                    731.0,
                    562.0,
                    733.0,
                    562.0,
                    734.0,
                    563.0,
                    734.0,
                    565.0,
                    736.0,
                    567.0,
                    737.0,
                    567.0,
                    738.0,
                    568.0,
                    740.0,
                    568.0,
                    741.0,
                    569.0,
                    743.0,
                    569.0,
                    748.0,
                    574.0,
                    748.0,
                    576.0,
                    749.0,
                    577.0,
                    749.0,
                    578.0,
                    746.0,
                    581.0,
                    745.0,
                    581.0,
                    742.0,
                    584.0,
                    741.0,
                    584.0,
                    739.0,
                    586.0,
                    738.0,
                    586.0,
                    737.0,
                    587.0,
                    736.0,
                    587.0,
                    735.0,
                    588.0,
                    734.0,
                    588.0,
                    733.0,
                    589.0,
                    732.0,
                    589.0,
                    731.0,
                    590.0,
                    730.0,
                    590.0,
                    729.0,
                    591.0,
                    728.0,
                    591.0,
                    727.0,
                    592.0,
                    727.0,
                    601.0,
                    740.0,
                    601.0,
                    741.0,
                    600.0,
                    742.0,
                    601.0,
                    785.0,
                    601.0,
                    786.0,
                    600.0,
                    787.0,
                    600.0,
                    788.0,
                    601.0,
                    793.0,
                    601.0,
                    794.0,
                    600.0,
                    799.0,
                    600.0,
                    800.0,
                    599.0,
                    802.0,
                    599.0,
                    802.0,
                    586.0,
                    798.0,
                    586.0,
                    797.0,
                    587.0,
                    793.0,
                    587.0,
                    792.0,
                    588.0,
                    789.0,
                    588.0,
                    788.0,
                    589.0,
                    783.0,
                    589.0,
                    782.0,
                    590.0,
                    781.0,
                    590.0,
                    780.0,
                    591.0,
                    778.0,
                    591.0,
                    777.0,
                    590.0,
                    778.0,
                    589.0,
                    778.0,
                    588.0,
                    779.0,
                    587.0,
                    780.0,
                    587.0,
                    782.0,
                    585.0,
                    783.0,
                    585.0,
                    784.0,
                    584.0,
                    784.0,
                    583.0,
                    785.0,
                    582.0,
                    786.0,
                    582.0,
                    787.0,
                    581.0,
                    788.0,
                    581.0,
                    789.0,
                    580.0,
                    790.0,
                    580.0,
                    791.0,
                    579.0,
                    792.0,
                    579.0,
                    798.0,
                    573.0,
                    799.0,
                    573.0,
                    801.0,
                    571.0,
                    802.0,
                    571.0,
                    802.0,
                    560.0,
                    783.0,
                    560.0,
                    783.0,
                    561.0,
                    781.0,
                    563.0,
                    780.0,
                    563.0,
                    778.0,
                    565.0,
                    777.0,
                    565.0,
                    776.0,
                    566.0,
                    774.0,
                    566.0,
                    773.0,
                    567.0,
                    770.0,
                    567.0,
                    769.0,
                    566.0,
                    764.0,
                    566.0,
                    763.0,
                    565.0,
                    763.0,
                    564.0,
                    761.0,
                    564.0,
                    760.0,
                    565.0,
                    759.0,
                    564.0,
                    756.0,
                    564.0,
                    755.0,
                    563.0,
                    755.0,
                    562.0,
                    756.0,
                    561.0,
                    756.0,
                    560.0,
                    734.0,
                    560.0,
                    733.0,
                    561.0,
                    732.0,
                    560.0
                ]

imgsz = image.shape[:2]
polygon = np.array(
    polygon_scaled,
)



mask = polygon2mask(
    imgsz,     # tuple
    [polygon], # input as list
    color=255, # 8-bit binary
    downsample_ratio=1
)


# Отобразить изображение и сегментированное изображение в консоли
cv2.imshow('212', image)
cv2.imshow('1212', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()